# Вебинар 3. Коллаборативная фильтрация

# ДЗ

1) Попытаться ответить на вопросы/выдвинуть гипотезы (в конце ноутбука)

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

## Практика:

4) Поэкспериментировать с ALS (grid-search)

# 4. Поэкспериментировать с ALS (grid-search)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [3]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# ALS

In [10]:
def fit_model(factors=64, regularization=0.05, u_i_matrix='original'):

    all_variants_matrix = {
        'original' : user_item_matrix,
        'tfidf': tfidf_weight(user_item_matrix.T).T,
        'bm25': bm25_weight(user_item_matrix.T).T
    }

    model = AlternatingLeastSquares(factors=factors,
                                    regularization=regularization,
                                    iterations=15,
                                    calculate_training_loss=True,
                                    num_threads=4,
                                    use_gpu=False)

    model.fit(csr_matrix(all_variants_matrix[u_i_matrix]).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    return model

In [11]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [12]:
def grid_search(factors=None, regularizations=None, user_item_matrixes=None):
    global result
    if factors is None:
        factors = [64]
    if regularizations is None:
        regularizations = [0.05]
    if user_item_matrixes is None:
        user_item_matrixes = ['original']
    scores = pd.DataFrame()
    for factor in factors:
        for regularization in regularizations:
            for matrix in user_item_matrixes:

                name_test = f'matrix: {matrix}, factor: {factor}, regularization: {regularization}'

                model = fit_model(factors=factor, regularization=regularization, u_i_matrix=matrix)

                result[name_test] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

                score = result.apply(lambda row: precision_at_k(row[name_test], row['actual']), axis=1).mean()
                scores = scores.append(pd.DataFrame([score], columns=['score'], index=[name_test]))
                print('score:', round(score,4), name_test)

    return result, scores.sort_values(by='score', ascending=False)

In [13]:
%%time

result, scores = grid_search(factors=[32, 64, 128],
                             regularizations=[0.01, 0.03, 0.05],
                             user_item_matrixes=['original', 'tfidf', 'bm25'])

  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1601 matrix: original, factor: 32, regularization: 0.01


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1672 matrix: tfidf, factor: 32, regularization: 0.01


  0%|          | 0/15 [00:00<?, ?it/s]

...

score: 0.1489 matrix: original, factor: 128, regularization: 0.05


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1517 matrix: tfidf, factor: 128, regularization: 0.05


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1918 matrix: bm25, factor: 128, regularization: 0.05
CPU times: user 11min 41s, sys: 36.4 s, total: 12min 18s
Wall time: 12min 22s


In [14]:
scores

,score
"matrix: bm25, factor: 128, regularization: 0.01",0.193571
"matrix: bm25, factor: 128, regularization: 0.05",0.191763
"matrix: bm25, factor: 128, regularization: 0.03",0.189955
"matrix: bm25, factor: 64, regularization: 0.05",0.174485
"matrix: bm25, factor: 64, regularization: 0.01",0.173682
"matrix: bm25, factor: 64, regularization: 0.03",0.173380
"matrix: tfidf, factor: 32, regularization: 0.05",0.167152
"matrix: tfidf, factor: 32, regularization: 0.01",0.167152
"matrix: tfidf, factor: 32, regularization: 0.03",0.166851
"matrix: original, factor: 32, regularization: 0.05",0.163938


In [15]:
result.head(2)

,user_id,actual,"matrix: original, factor: 32, regularization: 0.01","matrix: tfidf, factor: 32, regularization: 0.01","matrix: bm25, factor: 32, regularization: 0.01","matrix: original, factor: 32, regularization: 0.03","matrix: tfidf, factor: 32, regularization: 0.03","matrix: bm25, factor: 32, regularization: 0.03","matrix: original, factor: 32, regularization: 0.05","matrix: tfidf, factor: 32, regularization: 0.05",...,"matrix: bm25, factor: 64, regularization: 0.05","matrix: original, factor: 128, regularization: 0.01","matrix: tfidf, factor: 128, regularization: 0.01","matrix: bm25, factor: 128, regularization: 0.01","matrix: original, factor: 128, regularization: 0.03","matrix: tfidf, factor: 128, regularization: 0.03","matrix: bm25, factor: 128, regularization: 0.03","matrix: original, factor: 128, regularization: 0.05","matrix: tfidf, factor: 128, regularization: 0.05","matrix: bm25, factor: 128, regularization: 0.05"
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[885290, 1005186, 5569374, 999999, 995242]","[999999, 1082185, 1100972, 995242, 1005186]","[999999, 1082185, 1004390, 995242, 885290]","[885290, 1005186, 5569374, 999999, 832678]","[1100972, 999999, 1051211, 1082185, 995242]","[999999, 1082185, 995242, 885290, 981760]","[5569374, 1005186, 832678, 999999, 1082185]","[999999, 1082185, 995242, 1100972, 1024306]",...,"[999999, 1082185, 995242, 1051211, 1100972]","[1033142, 979707, 962568, 995242, 865178]","[878996, 962568, 1105488, 1033142, 1082185]","[999999, 995242, 1100972, 1082185, 934369]","[1033142, 1082212, 965766, 878996, 995242]","[965766, 962568, 995242, 1033142, 1024306]","[999999, 1082185, 995242, 965766, 15926844]","[1105488, 1033142, 979707, 995242, 5569374]","[1033142, 1043301, 1082185, 999999, 962568]","[999999, 995242, 1082185, 965766, 856942]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[951590, 883404, 999999, 908531, 1092026]","[951590, 999999, 962229, 1029743, 910032]","[951590, 999999, 840601, 883404, 8090521]","[951590, 962229, 883404, 1106523, 908531]","[999999, 951590, 962229, 1029743, 1082185]","[951590, 999999, 962229, 883404, 1029743]","[5569230, 962229, 951590, 883404, 8090521]","[951590, 999999, 1092026, 5569230, 844165]",...,"[999999, 951590, 1092026, 856772, 1133018]","[914190, 908531, 1133018, 938700, 5568378]","[1106523, 1022003, 1133018, 951590, 1092026]","[1133018, 999999, 1053690, 951590, 1092026]","[1133018, 1106523, 914190, 910032, 951590]","[1022003, 1133018, 1106523, 951590, 1042438]","[1133018, 999999, 1092026, 1053690, 908531]","[914190, 5568378, 1106523, 1133018, 1092026]","[951590, 1133018, 1106523, 883404, 1075470]","[999999, 951590, 1092026, 1133018, 5569230]"


### Итак, лучшую метрику 0.1936 получили при гипермараметрах: matrix: bm25, factor: 128, regularization: 0.01

# Попытаемся ответить на вопросы/выдвинуть гипотезы

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

# Links

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/